In [11]:
import numpy as np
import pandas as pd
from scipy.special import expit as sigmoid
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

In [12]:
# ==========================================================
# 1. Sigmoid + GP 混合 DDM 参数生成器
# ==========================================================
class HybridDDMParameterGenerator:
    def __init__(self, w=0.5):
        """
        w: 混合权重 (0~1)
        """
        self.w = w

        kernel = 1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1e-5)

        # GP 模型
        self.gp_v = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
        self.gp_a = GaussianProcessRegressor(kernel=kernel, normalize_y=True)

        # Sigmoid 的基础参数
        self.beta_v = np.array([0.01, 0.02, -0.01])
        self.beta_a = np.array([0.005, -0.01, 0.015])

    def sigmoid_part(self, X, beta):
        return sigmoid(X @ beta)

    def fit_gp(self, X, Y_v, Y_a):
        self.gp_v.fit(X, Y_v)
        self.gp_a.fit(X, Y_a)

    def predict_params(self, X):
        """返回 v, a, t0, z"""
        sig_v = self.sigmoid_part(X, self.beta_v)
        sig_a = self.sigmoid_part(X, self.beta_a)

        gp_v = self.gp_v.predict(X)
        gp_a = self.gp_a.predict(X)

        v = self.w * sig_v + (1 - self.w) * gp_v
        a = self.w * sig_a + (1 - self.w) * gp_a

        t0 = np.full(len(v), 0.2)       # 固定
        z = 2 / a                      # 固定关系

        return v, a, t0, z

In [13]:
# ==========================================================
# 2. Euler 马尔可夫链仿真 DDM
# ==========================================================
def simulate_ddm_euler(v, a, z, t0, dt=0.001):
    """
    返回 RT, response(1=上界, 0=下界)
    """
    x = z
    t = 0.0

    # 漂移扩散过程
    while 0 < x < a:
        dx = v * dt + np.sqrt(dt) * np.random.randn()
        x += dx
        t += dt

    RT = t + t0

    response = 1 if x >= a else 0
    return RT, response

In [20]:
# ==========================================================
# 3. 生成数据 + 保存 CSV
# ==========================================================
def generate_dataset(n_subjects=5, trials_per_sub=50, w=0.5, save_path="hybrid_ddm.csv"):
    gen = HybridDDMParameterGenerator(w=w)

    # ---- 训练 GP：使用随机点作为锚点 ----
    X_train = np.random.uniform(-1, 1, size=(50, 3))
    Y_v = np.sin(X_train[:, 0]) * 0.5 + 0.1 * np.random.randn(50)
    Y_a = 1.5 + 0.3 * np.cos(X_train[:, 1]) + 0.05 * np.random.randn(50)

    gen.fit_gp(X_train, Y_v, Y_a)

    # ---- 保存数据 ----
    rows = []

    for sub in range(1, n_subjects + 1):
        for trial in range(1, trials_per_sub + 1):

            # 输入变量
            P = np.random.uniform(-1, 1)
            T = np.random.uniform(-1, 1)
            W = np.random.uniform(-1, 1)

            X = np.array([[P, T, W]])
            v, a, t0, z = gen.predict_params(X)

            # 模拟 DDM
            RT, response = simulate_ddm_euler(v[0], a[0], z[0], t0[0])

            # M, label（你可能替换为真实实验变量）
            M = np.random.choice([0, 1])
            label = "Self" if M == 1 else "Other"
            
            rows.append([
                sub, trial, P, T, W, M, label,
                v[0], a[0], t0[0], z[0],
                RT, response
            ])

    df = pd.DataFrame(rows, columns=[
        "subject", "trial", "P", "T", "W", "M", "label",
        "v", "a", "t0", "z",
        "RT", "response"
    ])

    df.to_csv(save_path, index=False)
    print(f"✔ 数据已保存到 {save_path}")

    return df

In [21]:
# ==========================================================
# 运行
# ==========================================================
if __name__ == "__main__":
    df = generate_dataset(
        n_subjects=1,
        trials_per_sub=260,
        w=0.6,      #w = 0.5 → Sigmoid 和 GP 各占一半；w > 0.5 Sigmoid 占比更大；w < 0.5 GP 占比更大
        save_path="hybrid_ddm_output.csv"
    )
    print(df.head())


✔ 数据已保存到 hybrid_ddm_output.csv
   subject  trial         P         T         W  M  label         v         a  \
0        1      1 -0.624891  0.008516  0.465496  1   Self  0.298478  0.999675   
1        1      2 -0.857823  0.721609 -0.210554  0  Other  0.301282  0.996910   
2        1      3 -0.201793  0.593164 -0.343467  0  Other  0.302080  0.997295   
3        1      4  0.317388 -0.881627 -0.059358  0  Other  0.298008  1.000536   
4        1      5  0.497949 -0.911819  0.098651  1   Self  0.297952  1.001072   

    t0         z   RT  response  
0  0.2  2.000650  0.2         1  
1  0.2  2.006200  0.2         1  
2  0.2  2.005424  0.2         1  
3  0.2  1.998928  0.2         1  
4  0.2  1.997857  0.2         1  


c:\Users\蔡振辛\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\蔡振辛\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
